## Pydantic Model Dump


#### 1. Basic Usage


In [ ]:
from pydantic import BaseModel
from datetime import datetime


class LogEntry(BaseModel):
    id: int
    message: str
    timestamp: datetime

In [2]:
# 1. Create a model instance
log_data = {
    "id": 1,
    "message": "User login attempt successful.",
    "timestamp": datetime.now(),
}
entry = LogEntry(**log_data)

In [3]:
# --- model_dump(): Returns a Python Dictionary ---
print("--- 1. Using model_dump() ---")

data_dict = entry.model_dump()
print(f"Type of dump: {type(data_dict)}")
print(f"Dump Content:\n{data_dict}")

--- 1. Using model_dump() ---
Type of dump: <class 'dict'>
Dump Content:
{'id': 1, 'message': 'User login attempt successful.', 'timestamp': datetime.datetime(2025, 12, 9, 21, 32, 51, 355131)}


In [4]:
# --- model_dump_json(): Returns a JSON String ---
print("--- 2. Using model_dump_json() ---")

data_json = entry.model_dump_json(indent=2)  # Use indent for readability
print(f"Type of dump: {type(data_json)}")
print(f"Dump Content:\n{data_json}")

--- 2. Using model_dump_json() ---
Type of dump: <class 'str'>
Dump Content:
{
  "id": 1,
  "message": "User login attempt successful.",
  "timestamp": "2025-12-09T21:32:51.355131"
}


---

#### 2. Controlling Fields with include and exclude


In [5]:
from pydantic import BaseModel


class UserProfile(BaseModel):
    user_id: int
    username: str
    email: str
    password_hash: str  # Sensitive field
    is_active: bool

In [6]:
user = UserProfile(
    user_id=456,
    username="johnny_app",
    email="john@example.com",
    password_hash="a1b2c3d4e5",
    is_active=True,
)

In [7]:
# --- 1. Exclude Sensitive Fields ---
print("--- 1. Excluding Sensitive Data (Privacy) ---")

# Use exclude to omit one or more fields
public_data = user.model_dump(exclude={"password_hash", "email"})
print(f"Public Data: {public_data}")

--- 1. Excluding Sensitive Data (Privacy) ---
Public Data: {'user_id': 456, 'username': 'johnny_app', 'is_active': True}


In [8]:
# --- 2. Include Only Specific Fields ---
print("--- 2. Including Only Specific Fields (Summary) ---")

# Use include to specify the exact fields you want
summary_data = user.model_dump(include={"user_id", "username"})
print(f"Summary Data: {summary_data}")

--- 2. Including Only Specific Fields (Summary) ---
Summary Data: {'user_id': 456, 'username': 'johnny_app'}


---

#### 3. Handling Aliases and Defaults


In [9]:
from pydantic import BaseModel, Field


class Item(BaseModel):
    # 'item_id' is the model field name
    # 'id' is the input/output key (alias)
    item_id: int = Field(alias="id")

    name: str
    quantity: int = 1  # Default value

In [10]:
# Create an instance, but don't provide 'quantity'
item_data = {"id": 100, "name": "Notebook"}
item = Item(**item_data)

In [11]:
# --- 1. Dumping without Alias ---
print("--- 1. model_dump() without alias (Default) ---")

# Default output uses the internal model field names ('item_id', 'name', 'quantity')
internal_data = item.model_dump()
print(f"Internal Data: {internal_data}")

--- 1. model_dump() without alias (Default) ---
Internal Data: {'item_id': 100, 'name': 'Notebook', 'quantity': 1}


In [12]:
# --- 2. Dumping with Alias ---
print("--- 2. model_dump() with by_alias=True ---")

# Output keys use the defined alias ('id')
aliased_data = item.model_dump(by_alias=True)
print(f"Aliased Data: {aliased_data}")

--- 2. model_dump() with by_alias=True ---
Aliased Data: {'id': 100, 'name': 'Notebook', 'quantity': 1}


In [13]:
# --- 3. Dumping excluding defaults ---
print("--- 3. Excluding Default Values ---")

# Use exclude_defaults=True to omit fields that took their default value
no_defaults = item.model_dump(exclude_defaults=True)
print(f"No Defaults Data: {no_defaults}")

--- 3. Excluding Default Values ---
No Defaults Data: {'item_id': 100, 'name': 'Notebook'}


---

#### 4. Nested Models and Export Arguments


In [14]:
from pydantic import BaseModel
from typing import List


# Nested Model
class Permission(BaseModel):
    name: str
    level: int


# Main Model
class Group(BaseModel):
    group_name: str
    group_id: int
    permissions: List[Permission]

In [15]:
group_data = {
    "group_name": "Admins",
    "group_id": 1,
    "permissions": [
        {"name": "read", "level": 1},
        {"name": "write", "level": 2},
        {"name": "delete", "level": 3},
    ],
}
group = Group(**group_data)

In [16]:
# --- 1. Deep Exclusion (Excluding a field inside the nested model) ---
print("--- 1. Deep Exclusion (Exclude Permission Level) ---")

# Use the nested structure to target the 'level' field inside 'permissions'
exclude_nested = group.model_dump(
    exclude={
        "permissions": {
            "__all__": {"level"},
        },
    }
)
print(exclude_nested)

--- 1. Deep Exclusion (Exclude Permission Level) ---
{'group_name': 'Admins', 'group_id': 1, 'permissions': [{'name': 'read'}, {'name': 'write'}, {'name': 'delete'}]}


In [17]:
# --- 2. Excluding the Entire Nested List ---
print("--- 2. Excluding the Entire Nested List ---")

# Simply exclude the field name
exclude_list = group.model_dump(exclude={"permissions"})
print(exclude_list)

--- 2. Excluding the Entire Nested List ---
{'group_name': 'Admins', 'group_id': 1}
